# Example to connect to postgres

This is a tutorial to connect to our PostgreSQL database using python.

First you need to install the following libraries:
- psycopg2
- python-dotenv

Then you need to create a file with the name ".env". This file will contain the connection information and your credentials. This is an example:

```
DB_HOST=host_name
DB_NAME=postgres
DB_USER=my_user
DB_PASSWORD=my_password
DB_PORT=5432
```

After that you´re all set. We will import your credentials and connect to the database.

In [ ]:
import os
import psycopg2 # PostgreSQL database adapter for Python
from dotenv import load_dotenv # Reads the key-value pair from .env file and adds them to environment variable

# Load environment variables from .env file
load_dotenv()

# Accessing credentials
db_host = os.getenv("DB_HOST")
db_name = os.getenv("DB_NAME")
db_user = os.getenv("DB_USER")
db_password = os.getenv("DB_PASSWORD")
db_port = os.getenv("DB_PORT")

In [ ]:
# Connect to the database
conn = psycopg2.connect(
    host=db_host,
    dbname=db_name,
    user=db_user,
    password=db_password,
    port=db_port
)

Now we can query our data (write sql code) and store it as a pandas dataframe

In [ ]:
import pandas as pd
from sklearn.metrics import mean_squared_error
import matplotlib.pyplot as plt

In [ ]:
query="""
    select * 
    from agg.tidy_data_final  
    where year between 2019 and 2020
"""

In [ ]:
df = pd.read_sql_query(query, conn)

In [ ]:
df.info()


In [ ]:
# Average hourly
from sklearn.preprocessing import MinMaxScaler
df = df.drop('timestamp', axis=1)
#df = df.groupby(['year','month', 'day', 'hour','minute']).mean().reset_index()
#df['net_load_norm'] = df['net_load'] / max(abs(df['net_load']))
df['datetime'] = pd.to_datetime(df[['year','month', 'day', 'hour','minute']])
df = df.sort_values(by='datetime')


In [ ]:
columns_to_keep = ['net_load', 'datetime','solar_radiation','sunshine_duration','precipitation_probability','site','weekend_or_bank_holiday','hour']

# Keep only the specified columns
df = df.loc[:, columns_to_keep]
df['net_load'] = df['net_load'].astype('float32')
df['solar_radiation'] = df['solar_radiation'].astype('float32')
df['sunshine_duration'] = df['sunshine_duration'].astype('float32')
df['precipitation_probability'] = df['precipitation_probability'].astype('float32')

def normalize_net_load(group):
    max_net_load = abs(group['net_load']).max()  # Get maximum net load value for the site
    group['net_load_norm'] = group['net_load'] / max_net_load  # Calculate normalized net load
    return group

# Apply normalization function to each site group using transform()
df['net_load_n2'] = df.groupby('site')['net_load'].transform(lambda x: x / abs(x).max())



print(df)

df.head()

In [ ]:
site_20_data = df[df['site'] == 6]
plt.plot(site_20_data['datetime'], site_20_data['net_load_n2'], label='Net Load (Site 20)', color='b')

In [ ]:
df = df[df['datetime'] >= '2019-11-25'] # df = df[df['datetime'] >= '2020-1-25']
df = df[df['datetime'] <= '2020-3-26']

In [ ]:
avg_load_day = df.groupby(df['datetime'].dt.date)['net_load_n2'].mean()
plt.plot(avg_load_day)
plt.xticks(rotation=45) 

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from statsmodels.tsa.arima.model import ARIMA
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
df = df.sort_values(by='datetime')

# Plot the time series data
plt.plot(df['datetime'],df['net_load'])
plt.xlabel('Datetime')
plt.ylabel('net_load_norm')
plt.title('Time Series Data')
plt.show()

# Plot autocorrelation and partial autocorrelation plots
'''
plot_acf(df['net_load'])
plt.title('Autocorrelation Plot')
plt.show()

plot_pacf(df['net_load'])
plt.title('Partial Autocorrelation Plot')
plt.show()
'''

In [ ]:
from statsmodels.tsa.statespace.sarimax import SARIMAX
import numpy as np
import matplotlib.pyplot as plt
# Run the arim model and get the equation should speed everthing up and also be more complex
'''
df = df.sort_values(by='datetime')
results = pd.DataFrame()
for x in range(4,0,-1):
    vals = x*-24
    train_data = df.iloc[:vals]  
    test_data = df.iloc[vals:]
    test_data = test_data.iloc[:24]

    model = SARIMAX(train_data['net_load_norm'], exog=train_data[['solar_radiation','weekend_or_bank_holiday']], order=(4, 1, 3), seasonal_order=(2, 1, 1, 24))
    sarimax_model = model.fit()

    sarimax_params = sarimax_model.params

    print("SARIMAX Parameters:")
    print(sarimax_params)

    forecast_horizon = 24
    forecast = sarimax_model.forecast(steps=forecast_horizon, exog=test_data[['solar_radiation','weekend_or_bank_holiday']])
    #print('Forecasted Values:', forecast)
    df_final = pd.DataFrame()
    df_final = pd.concat([test_data, forecast], axis=1)
    print(df_final)

    results = pd.concat([results, df_final],ignore_index=True)

results = results.rename(columns={results.columns[-1]: 'forecast'})

plt.figure(figsize=(10, 6))
plt.plot(df['datetime'], df['net_load_norm'], label='Observed')
plt.plot(results['datetime'], results['forecast'], label='Forecast', color='red')
plt.xlabel('Date')
plt.ylabel('Value')
plt.title('SARIMAX Model Forecast')
plt.legend()
plt.xticks(rotation=45) 
plt.grid(alpha=0.3)
plt.show()
'''

In [ ]:
'''
df2 = df[-300:]
plt.figure(figsize=(10, 6))
plt.plot(df2['datetime'], df2['net_load'], label='Observed')
plt.plot(results['datetime'], results['forecast'], label='Forecast', color='red')
plt.xlabel('Date')
plt.ylabel('Value')
plt.title('SARIMAX Model Forecast')
plt.legend()
plt.xticks(rotation=45) 
plt.grid(alpha=0.3)
plt.show()
'''

In [ ]:
import numpy as np
import pandas as pd
import pmdarima as pm
from tqdm import tqdm
'''
# Run the auto arima get the params then run the non auto with the params for each it

# Assuming 'df' is your dataframe containing the data
train_data = df.iloc[:-336]

# Define ARIMA parameters ( need to trial q = 5)
start_p = 1  
start_d = 1  
start_q = 1  
max_p = 3  
max_d = 2  
max_q = 3  

start_P = 1  
start_D = 1  
start_Q = 2  
max_P = 1  
max_D = 1  
max_Q = 3  


# Initialize tqdm progress bar
with tqdm(total=1) as pbar:
    def progress_callback(iteration):
        pbar.update(1)

    # Train ARIMA model with progress callback
    model_auto = pm.auto_arima(
        train_data['net_load_norm'],
        exogenous=train_data[['solar_radiation','sunshine_duration', 'precipitation_probability']],
        seasonal=True,
        m=48,
        start_p=start_p,
        start_d=start_d,
        start_q=start_q,
        max_d=max_d,
        max_q=max_q,
        start_P=start_P,
        start_D=start_D,
        start_Q=start_Q,
        max_D=max_D,
        max_Q=max_Q,
        stepwise=True,
        method='nm',
        maxiter=50,
        trace=True,
        callback=progress_callback
    )
    '''

In [ ]:
def assign_daytime(hour):
    if hour <= 6:
        return 1
    else:
        return 0
def assign_daytime2(hour):
    if 6 < hour <= 10:
        return 1
    else:
        return 0
def assign_daytime3(hour):
    if 10 < hour <= 16:
        return 1
    else:
        return 0
def assign_daytime4(hour):
    if 16 < hour <= 21:
        return 1
    else:
        return 0
def assign_daytime5(hour):
    if 21 < hour:
        return 1
    else:
        return 0

df['time1'] = df['hour'].apply(assign_daytime)
df['time2'] = df['hour'].apply(assign_daytime2)
df['time3'] = df['hour'].apply(assign_daytime3)
df['time4'] = df['hour'].apply(assign_daytime4)
df['time5'] = df['hour'].apply(assign_daytime5)

'''
def assign_daytime(hour):
    if hour <= 2:
        return 1
    else:
        return 0
def assign_daytime2(hour):
    if 2 < hour <= 4:
        return 1
    else:
        return 0
def assign_daytime3(hour):
    if 4 < hour <= 6:
        return 1
    else:
        return 0
def assign_daytime4(hour):
    if 6 < hour <= 8:
        return 1
    else:
        return 0
def assign_daytime5(hour):
    if 8 < hour <= 10:
        return 1
    else:
        return 0

def assign_daytime6(hour):
    if 10 < hour <= 12:
        return 1
    else:
        return 0
def assign_daytime7(hour):
    if 12 < hour <= 14:
        return 1
    else:
        return 0
def assign_daytime8(hour):
    if 14 < hour <= 16:
        return 1
    else:
        return 0
def assign_daytime9(hour):
    if 16 < hour <= 18:
        return 1
    else:
        return 0
def assign_daytime10(hour):
    if 18 < hour <= 20:
        return 1
    else:
        return 0
def assign_daytime11(hour):
    if 20 < hour <= 22:
        return 1
    else:
        return 0
def assign_daytime12(hour):
    if 22 < hour:
        return 1
    else:
        return 0

df['time1'] = df['hour'].apply(assign_daytime)
df['time2'] = df['hour'].apply(assign_daytime2)
df['time3'] = df['hour'].apply(assign_daytime3)
df['time4'] = df['hour'].apply(assign_daytime4)
df['time5'] = df['hour'].apply(assign_daytime5)
df['time6'] = df['hour'].apply(assign_daytime6)
df['time7'] = df['hour'].apply(assign_daytime7)
df['time8'] = df['hour'].apply(assign_daytime8)
df['time9'] = df['hour'].apply(assign_daytime9)
df['time10'] = df['hour'].apply(assign_daytime10)
df['time11'] = df['hour'].apply(assign_daytime11)
df['time12'] = df['hour'].apply(assign_daytime12)
'''

In [ ]:
def ARIMAorder(data):

    fit = pm.auto_arima(
        data['net_load_norm'],
        exogenous=data[['solar_radiation','sunshine_duration', 'precipitation_probability','weekend_or_bank_holiday']],
        seasonal=True,
        m=48,
        stepwise=True,
        method='nm',
        maxiter=50,
        trace=True,
        start_p=1,
        start_q=1,
        max_p=2,
        max_q=2,
        max_d=2,
        start_P=1,
        start_Q=1,
        max_P=2,
        max_Q=2,
        max_D=2,
    )
    return fit

In [ ]:
from sklearn.metrics import mean_absolute_error
'month','day','hour','isweekend','solar radiation', 'sunshine duration','precipitation probability'
'''    start_p=1,
        start_d=1,
        start_q=1,
        max_p=3,
        max_d=2,
        max_q=3,
        start_P=1,
        start_D=1,
        start_Q=1,
        max_P=2,
        max_D=2,
        max_Q=2,
        '''
def ARIMAorder(data):

    fit = pm.auto_arima(
        data['net_load_n2'],
        exogenous=data[['solar_radiation','sunshine_duration', 'precipitation_probability','weekend_or_bank_holiday','time1','time2','time3','time4','time5']],
        stepwise=True,
        method='nm',
        maxiter=100,
        trace=True,
        start_p=1,
        start_q=1,
        start_d=1,
        max_p=5,
        max_q=3,
        max_d=5
    )
    return fit

In [ ]:
# THE BIG LOOP
em = ['site','MAE','MSE']
em_df = pd.DataFrame(columns=em)
houses = [2, 3,6, 9, 11, 12, 16] 
#houses = [17, 20, 21, 22]
#houses = [23, 25] [28!!!!!] order = (1,0,1), seasonal_order=(2,0,1,48)) [29]
#houses = [30, 31, 33] #34, 36, 39,41, 42, 46, 49, 50, 53, 57, 61, 62, 64, 73, 77, 81, 84, 85, 90, 91, 94, 98, 100]

# loop for each site
for i in houses:
    site_df = df[df['site']==i]
    #site_df['net_load_norm'] = site_df['net_load'] / max(abs(site_df['net_load']))
    site_df_train = site_df[:-1464]
    print('House:',i)
    # run the auto arima and extract the optimal params 
    fit = ARIMAorder(site_df_train)
    results = pd.DataFrame()
    
    # Loop for each day
    for x in range(31,1,-1):
        # get the most up to date data for this predicton
        vals = -x*48 +24 #to get midday
        train_data = site_df.iloc[:vals]  
        test_data = site_df.iloc[vals:]
        test_data = test_data.iloc[:72]

        # Create the model with the new data
        model_auto = SARIMAX(train_data['net_load_n2'], exog=train_data[['solar_radiation','sunshine_duration', 'precipitation_probability','weekend_or_bank_holiday','time1','time2','time3','time4','time5']], order= fit.get_params().get("order"))#,seasonal_order=fit.get_params().get("seasonal_order"))
        sarimax_model = model_auto.fit()

        # Forecast the next period
        forecast_horizon = 72
        forecast = sarimax_model.forecast(steps=forecast_horizon, exog=test_data[['solar_radiation','sunshine_duration', 'precipitation_probability','weekend_or_bank_holiday','time1','time2','time3','time4','time5']])
        df_final = pd.DataFrame(test_data)
        forecast_df = pd.DataFrame(forecast)
        df_final['forecast'] = forecast_df['predicted_mean'].values
        df_final = df_final[24:]
        results = pd.concat([results, df_final],ignore_index=True)
        print(x-1)

    # add the evaluation metric for the site to a df
    norm_mae = round(mean_absolute_error(results['net_load_n2'], results['forecast']),4)
    norm_mse = round(mean_squared_error(results['net_load_n2'], results['forecast']),4)
    house_res = {'site': i, 'MAE': norm_mae, 'MSE': norm_mse}
    house_res_df = pd.DataFrame([house_res])
    em_df= pd.concat([em_df, house_res_df],ignore_index=True)


    # Plot the figure for each house 
    plt.figure(figsize=(10, 6))
    plt.plot(site_df['datetime'], site_df['net_load_n2'], label='Observed')
    plt.plot(results['datetime'], results['forecast'], label='Forecast', color='red')
    plt.xlabel('Date')
    plt.ylabel('Value')
    plt.title('SARIMAX Model Forecast')
    plt.legend()
    plt.xticks(rotation=45) 
    plt.grid(alpha=0.3)
    plt.show()


In [ ]:
print(em_df)

In [ ]:
file_path = 'C:/Users/xlow6/MSc_ESDA/2.AML/em.csv'
# Save DataFrame to CSV
em_df.to_csv(file_path, index=False)

In [ ]:
df2 = site_df[site_df['datetime'] <= '2020-3-26']
df2 = df2[df2['datetime'] >= '2020-3-1']

results2 = results[results['datetime'] <= '2020-3-26']
results2 = results2[results2['datetime'] >= '2020-3-1']

plt.figure(figsize=(10, 6))
plt.plot(df2['datetime'], df2['net_load_norm'], label='Observed',alpha=0.6)
plt.plot(results2['datetime'], results2['forecast'], label='Forecast', color='red',alpha=1)
plt.xlabel('Date')
plt.ylabel('Value')
plt.title('SARIMAX Model Forecast')
plt.legend()
plt.xticks(rotation=45) 
plt.grid(alpha=0.3)

date_range1 = pd.date_range(start='2020-03-01', end='2020-03-06', freq='D')


# Plot each date with a vertical line at 12:00
for date in date_range1:
    plt.axvline(x=date + pd.Timedelta(hours=12), color='red', linestyle='--', alpha=0.5)
    plt.axvline(x=date, color='black', linestyle='--', alpha=0.5)


In [ ]:
print(site_df_train)

In [ ]:
mse = round(mean_squared_error(results['net_load_norm'], results['forecast']),4)
print(mse)

In [ ]:
## 36 Hour forecast
## Train model and save then load model and update each time 
'''
results = pd.DataFrame()
for x in range(30,1,-1):
    vals = -x*48 +24 #to get midday
    train_data = df.iloc[:vals]  
    test_data = df.iloc[vals:]
    test_data = test_data.iloc[:72]

    #model_auto = model_auto.update(train_data['net_load_norm'], exog=train_data[['solar_radiation','sunshine_duration', 'precipitation_probability']], maxiter=1,trace=True,inplace=True)
    #model_auto.update(train_data['net_load_norm'])

    model_auto = SARIMAX(train_data['net_load_norm'], exog=train_data[['solar_radiation','sunshine_duration', 'precipitation_probability']], order=(1, 1, 1), seasonal_order=(1, 0, 0, 48))
    sarimax_model = model_auto.fit()

    #sarimax_params = model_auto.params

    #print("SARIMAX Parameters:")
    #print(sarimax_params)

    forecast_horizon = 72
    forecast = sarimax_model.forecast(steps=forecast_horizon, exog=test_data[['solar_radiation','sunshine_duration', 'precipitation_probability']])
    #forecast, conf_int = model_auto.predict(n_periods=72, exogenous=test_data[['solar_radiation','sunshine_duration', 'precipitation_probability']], return_conf_int=True, order=(1, 1, 1), seasonal_order=(1, 0, 0, 48))
    #forecast = model_auto.predict(n_periods=72, exogenous=test_data[['solar_radiation','sunshine_duration', 'precipitation_probability']], return_conf_int=True, order=(1, 1, 1), seasonal_order=(1, 0, 0, 48))
    df_final = pd.DataFrame(test_data)
    forecast_df = pd.DataFrame(forecast)
    #print(df_final)
    #print(forecast_df)
    df_final['forecast'] = forecast_df['predicted_mean'].values
    df_final = df_final[24:]
    #print(conf_int)

    results = pd.concat([results, df_final],ignore_index=True)
    print(x-1)

#results = results.rename(columns={results.columns[-1]: 'forecast'})

plt.figure(figsize=(10, 6))
plt.plot(df['datetime'], df['net_load_norm'], label='Observed')
plt.plot(results['datetime'], results['forecast'], label='Forecast', color='red')
plt.xlabel('Date')
plt.ylabel('Value')
plt.title('SARIMAX Model Forecast')
plt.legend()
plt.xticks(rotation=45) 
plt.grid(alpha=0.3)
plt.show()
#INdexing and setting params in prediction line
'''

In [ ]:
'''
## Day Forecast
results = pd.DataFrame()
for x in range(4,0,-1):
    vals = -x*48
    train_data = df.iloc[:vals]  
    test_data = df.iloc[vals:]
    test_data = test_data.iloc[:48]

    model_auto.update(train_data['net_load_norm'], exog=train_data[['solar_radiation','weekend_or_bank_holiday','sunshine_duration', 'precipitation_probability']], maxiter=1)
    #model_auto.update(train_data['net_load_norm'])

    sarimax_params = model_auto.params

    print("SARIMAX Parameters:")
    print(sarimax_params)

    #forecast_horizon = 24
    #forecast = model.forecast(steps=forecast_horizon, exog=test_data[['solar_radiation','weekend_or_bank_holiday']])
    forecast, conf_int = model_auto.predict(n_periods=48, exogenous=test_data[['solar_radiation','weekend_or_bank_holiday','sunshine_duration', 'precipitation_probability']], return_conf_int=True)
    df_final = pd.DataFrame(test_data)
    df_final['forecast'] = forecast 

    results = pd.concat([results, df_final],ignore_index=True)
    print(x)

results = results.rename(columns={results.columns[-1]: 'forecast'})

plt.figure(figsize=(10, 6))
plt.plot(df['datetime'], df['net_load_norm'], label='Observed')
plt.plot(results['datetime'], results['forecast'], label='Forecast', color='red')
plt.xlabel('Date')
plt.ylabel('Value')
plt.title('SARIMAX Model Forecast')
plt.legend()
plt.xticks(rotation=45) 
plt.grid(alpha=0.3)
plt.show()
'''

In [ ]:
df2 = df[-1440:]
plt.figure(figsize=(10, 6))
plt.plot(df2['datetime'], df2['net_load_norm'], label='Observed',alpha=0.6)
plt.plot(results['datetime'], results['forecast'], label='Forecast', color='red',alpha=1)
plt.xlabel('Date')
plt.ylabel('Value')
plt.title('SARIMAX Model Forecast')
plt.legend()
plt.xticks(rotation=45) 
plt.grid(alpha=0.3)

#plt.axvline(x=pd.to_datetime('2020-3-24 12:00:00'), color='black', linestyle='--',linewidth=1)

start_date = pd.Timestamp('2020-02-26')  # Replace with your desired start date
end_date = pd.Timestamp('2020-03-26')    # Replace with your desired end date
plt.xlim(start_date, end_date)
plt.ylim(-1,1)



plt.show()

In [ ]:
from sklearn.metrics import mean_absolute_error
norm_mae = mean_absolute_error(results['net_load_norm'], results['forecast'])
print(round(norm_mae,4))

def mean_absolute_arctangent_percentage_error(y_true, y_pred):
    y_true, y_pred = np.array(y_true), np.array(y_pred)
    return np.mean(np.abs(np.arctan((y_true - y_pred) / np.abs(y_true)) / (np.pi / 2)))

maape = mean_absolute_arctangent_percentage_error(results['net_load_norm'], results['forecast'])
print(round(maape,3))
